In [128]:
import pandas as pd
import csv
import re

In [149]:
#Creates a writeable csv for inputs
with open("In-DayX.csv","w") as Output:
    Output.write('ID Number'+","+'Time In'+","+'Time Out'+","+'Deductibles'+","+'Bonus'+"\n")

In [129]:
#Regular Info
Employee_Information_df = pd.read_csv("Employee-Information.csv")
Standard_Clock_Hours_df = pd.read_csv("Standard-Clock-Hours.csv")

#Input #Reads the csv #Please change the name to the name of input csv
Day_df = pd.read_csv("In-Monday.csv")

In [130]:
Employee_Information_df.columns

Index(['ID Number', 'First', 'Last', 'Date Started', 'Date Left',
       'Rate Per Day', 'Rate Per Month'],
      dtype='object')

In [150]:
Employee_Information_df

,ID Number,First,Last,Date Started,Date Left,Rate Per Day,Rate Per Month
0,10000000,Angelo,Johnson,12/06/2018,0,537,16110
1,10000001,Bea,Ramirez,30/07/2015,0,600,18000
2,10000002,Chris,Evans,22/12/2018,0,840,25200
3,10000003,Dennis,Tan,04/12/2016,0,550,16500
4,10000004,Emily,Santos,14/08/2017,0,1045,31350
5,10000005,Frances,Mercado,15/03/2019,0,729,21870
6,10000006,Gary,Cruz,21/03/2016,0,1255,37650
7,10000007,Hans,Chan,16/05/2018,0,920,27600
8,10000008,Isabel,Juan,18/05/2017,0,620,18600
9,10000009,Jen,Ang,12/09/2018,0,845,25350


In [132]:
Standard_Clock_Hours_df.columns

Index(['Date', 'Clock In Time', 'Clock Out Time'], dtype='object')

In [133]:
Standard_Clock_Hours_df

,Date,Clock In Time,Clock Out Time
0,Monday,8:00,16:00
1,Tuesday,8:00,16:00
2,Wednesday,8:00,16:00
3,Thursday,8:00,16:00
4,Friday,8:00,16:00
5,Saturday,8:00,16:00
6,Sunday,8:00,12:00


In [134]:
# Removes Colons
Day_df.loc[:,"Time In"] = Day_df.loc[:,"Time In"].apply(lambda x: re.sub(":","", x) )
Day_df.loc[:,"Time Out"] = Day_df.loc[:,"Time Out"].apply(lambda x: re.sub(":","", x) )

Standard_Clock_Hours_df.loc[:,"Clock In Time"] = Standard_Clock_Hours_df.loc[:,"Clock In Time"].apply(lambda x: re.sub(":","", x) )
Standard_Clock_Hours_df.loc[:,"Clock Out Time"] = Standard_Clock_Hours_df.loc[:,"Clock Out Time"].apply(lambda x: re.sub(":","", x) )

In [135]:
def time_to_minutes(x):
    if len(str(x))==3:
        hours = str(x)
        hours = int(hours[0])*60
        minutes = str(x)
        minutes = int(minutes[1:3])
        time = hours + minutes
        time = str(time)
        return time
    if len(str(x))==4:
        hours = str(x)
        hours = int(hours[0:2])*60
        minutes = str(x)
        minutes = int(minutes[2:4])
        time = hours + minutes
        time = str(time)
        return time
time_to_minutes("800")

'480'

In [136]:
def time_round_up(x):
    hours = int(x)/60
    rational = float(hours)
    integer = int(hours)
    decimal = rational - integer
    if decimal >=0.5:
        roundnumber = integer + 1
        return str(roundnumber)
    else:
        return str(integer)
time_round_up("800")

'13'

In [137]:
def work_hours(x):
    if int(x)<=8:
        work_hours = int(x)
        return work_hours
    else:
        work_hours = 8
        return work_hours
    return work_hours
work_hours(9)

8

In [138]:
def overtime_hours(x):
    if int(x)<=8:
        overtime_hours = 0
        return overtime_hours
    elif int(x)>8:
        overtime_hours = int(x) - 8
        return overtime_hours
    return overtime_hours
overtime_hours(10)

2

In [139]:
def late_hours(x):
    if int(x)<=0:
        late_hours = 0
        return late_hours
    elif int(x)>0:
        late_hours = int(x) / 60
        rational = float(late_hours)
        integer = int(late_hours)
        decimal = rational - integer
        if decimal >=0:
            late_hours = integer + 1
            return late_hours
        else:
            return integer
late_hours(1)

1

In [145]:
#Output #Remember to Rename Day
with open("Output_Reference.csv","w") as Output:
    Output.write('ID Number'+","+'TI min'+","+'TO min'+","+'Work Hours'+","+'Basic Salary'+","+'Overtime Hours'+","+'Overtime Pay'+","
               +'Late Minutes'+","+'Late Deductions'+","+'Deductibles'+","+'Bonus'+"\n")

Out_DayX_df_Initial = pd.read_csv("Output_Reference.csv")

#Merge Salary Information
Employee_Wages_df = Employee_Information_df.loc[:,["ID Number","Rate Per Day"]]
Out_DayX_df = pd.merge(Employee_Wages_df,Out_DayX_df_Initial,on="ID Number",how="left")

#Input Information
Out_DayX_df.loc[:,"ID Number"] = Day_df.loc[:,"ID Number"]
Out_DayX_df.loc[:,"TI min"] = Day_df.loc[:,"Time In"].apply(lambda x: time_to_minutes(x))
Out_DayX_df.loc[:,"TO min"] = Day_df.loc[:,"Time Out"].apply(lambda x: time_to_minutes(x))
#Basic Pay
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"TO min"].astype(int) - Out_DayX_df.loc[:,"TI min"].astype(int)
Out_DayX_df.loc[:,"Work Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: time_round_up(x))
Out_DayX_df.loc[:,"Basic Salary"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: work_hours(x)) * (Out_DayX_df.loc[:,"Rate Per Day"]/8)
#Overtime Pay
Out_DayX_df.loc[:,"Overtime Hours"] = Out_DayX_df.loc[:,"Work Hours"].apply(lambda x: overtime_hours(x))
Out_DayX_df.loc[:,"Overtime Pay"] = (Out_DayX_df.loc[:,"Overtime Hours"] * (Out_DayX_df.loc[:,"Rate Per Day"]/8) * 1.3).round(2)


#Lates                                       # (0:0) Monday Standard Clock In
Standard_Clock_In = Standard_Clock_Hours_df.loc[0:0,"Clock In Time"].apply(lambda x: time_to_minutes(x))
Stringone = str(Standard_Clock_In)
Stringone = re.sub(r"\D",r"",Stringone)
Stringone = int(Stringone)
Out_DayX_df.loc[:,"Late Minutes"] = Out_DayX_df.loc[:,"TI min"].astype(int) - Stringone
Out_DayX_df.loc[:,"Late Deductions"] = (Out_DayX_df.loc[:,"Late Minutes"].apply(lambda x: late_hours(x)) * (Out_DayX_df.loc[:,"Rate Per Day"]/8)).round(2)

#Miscellaneous
Out_DayX_df.loc[:,"Deductibles"] = Day_df.loc[:,"Deductibles"]
Out_DayX_df.loc[:,"Bonus"] = Day_df.loc[:,"Bonus"]

#Standing Salary
Out_DayX_df.loc[:,"Standing Salary"] = (Out_DayX_df.loc[:,"Basic Salary"] + Out_DayX_df.loc[:,"Overtime Pay"] - Out_DayX_df.loc[:,"Late Deductions"] - Out_DayX_df.loc[:,"Deductibles"] + Out_DayX_df.loc[:,"Bonus"]).round(2)

In [146]:
Out_DayX_df

,ID Number,Rate Per Day,TI min,TO min,Work Hours,Basic Salary,Overtime Hours,Overtime Pay,Late Minutes,Late Deductions,Deductibles,Bonus,Standing Salary
0,10000000,537,460,960,8,537.0,0,0.00,-20,0.00,0,0,537.00
1,10000001,600,480,982,8,600.0,0,0.00,0,0.00,220,0,380.00
2,10000002,840,490,1040,9,840.0,1,136.50,10,105.00,80,0,791.50
3,10000003,550,480,1080,10,550.0,2,178.75,0,0.00,0,0,728.75
4,10000004,1045,482,1010,9,1045.0,1,169.81,2,130.62,0,0,1084.19
5,10000005,729,473,960,8,729.0,0,0.00,-7,0.00,100,0,629.00
6,10000006,1255,475,1040,9,1255.0,1,203.94,-5,0.00,130,0,1328.94
7,10000007,920,489,982,8,920.0,0,0.00,9,115.00,120,0,685.00
8,10000008,620,565,1024,8,620.0,0,0.00,85,155.00,20,0,445.00
9,10000009,845,500,1102,10,845.0,2,274.62,20,105.62,100,0,914.00


In [147]:
                                                        # Change name of the output csv
Output_csv = Out_DayX_df.to_csv(r'C:\Users\ASUS\Desktop\Out-Monday.csv')